In [7]:
# !pip install langchain_groq langchain_community google-search-results
#using external tool langchain

  Using cached langchain_groq-0.3.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_search_results-2.4.2-py3-none-any.whl
  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.9.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached orjson-3.10.18-cp312-cp312-macosx_15_0_arm64.whl.metadata (41 kB)
  Using cached typing_extensions-4.14.0-py3-none-any.whl.metadata (3.0 kB)
Using cached langchain_groq-0.3.2-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.0 MB/s eta 0:00:00a 0:00:01
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using c

In [9]:
import os
from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent

In [ ]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SERP_API_KEY = os.getenv("SERP_API_KEY")

In [ ]:
# Initialize the ChatGroq LLM with the LLaMA3-70B model
groq_llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model_name="llama3-70b-8192", # This model first tries to answer using its own knowledge. 
                                   # If it can't answer (e.g., for time-sensitive questions), it can fall back to web search.
    temperature=1.0,               # Controls randomness; higher values make output more creative
    max_tokens=500                 # Limits the length of the generated response
)

# Initialize SerpAPI wrapper for performing real-time web searches
search=SerpAPIWrapper(serpapi_api_key=SERP_API_KEY)

# Define a tool for web searching using SerpAPI
search_tool = Tool(
    name="Web Search",
    func=search.run,                     # Function to execute the web search using the query
    description="Searches the web for latest information", # Used to decide when to call this tool
    handle_parsing_errors=True          # Prevents tool failure if there's an error during parsing
)

# memory=ConversationBufferMemory(memory_key = "history")

In [15]:
agent = initialize_agent(
    tools=[search_tool], # Tools available to the agent
    llm=groq_llm, # Language model used by the agent
    # memory=memory,
    verbose=True# Print intermediate steps
)

In [16]:
response = agent.invoke("Who is the current elected president of USA of 2025?")



> Entering new AgentExecutor chain...
Thought: I don't have real-time information, but I can try to search the web for information available up to a certain point in time.

Action: Web Search
Action Input: "current president of the United States" 

Observation: Donald Trump
Thought:Thought: Wait, I think I made a mistake. The observation is showing Donald Trump, but the question is asking about the president in 2025. I need to search for more specific information.

Action: Web Search
Action Input: "president of the United States 2025" 

Observation: [{'title': 'Department Press Briefing – June 17, 2025', 'link': 'https://www.state.gov/briefings/department-press-briefing-june-17-2025/', 'source': 'U.S. Department of State (.gov)', 'source_logo': 'https://serpapi.com/searches/68520bdacc56d38b9462157f/images/fa800f740c4b75232f4e79ba0b8e1a3dd47008695bee3a4e494031d1c89c8aef.png', 'date': '3 hours ago', 'thumbnail': 'https://serpapi.com/searches/68520bdacc56d38b9462157f/images/fa800f740c4b

In [17]:
print(response)

{'input': 'Who is the current elected president of USA of 2025?', 'output': 'The current elected president of the United States as of 2025 is still unknown based on the available information up to a certain point in time. The searches did not yield a definitive answer, and it appears that the information available is outdated or not specific enough to provide a clear answer.'}
